The result of CNN_catordog ver1 was not as good as we want.

We can make this NN deeper to improve our model's performance.

How to make it deeper?
###    There are two ways to do:
-    1. add the second convolutional layer
-    2. add another fully connected layer

# Part1 : Build a CNN

In [1]:
# import the keras libraries and packages
from keras.models import Sequential 
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Initialising the CNN
model = Sequential()

In [3]:
# Step1 : Convolution layer
model.add(Convolution2D(32,(3,3), input_shape=(64,64,3), activation='relu' ))


In [4]:
# Step2 : Pooling layer : in oder to reduce the numbers of input nodes before flatten this to a large vector
model.add(MaxPooling2D(pool_size=(2,2)))


#  ------- The second Convolutional layer -------

In [5]:
model.add(Convolution2D(32,(3,3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
# If have GPU, you can try Convolution2D(64, (3,3)...), and add the third Convolution Layer, then 128 ....

In [6]:
# Step3: Flatten
model.add(Flatten())


In [7]:
# Step4 : Fully connected layer as hidden layer
model.add(Dense(activation="relu", units=128))

# Output layer
model.add(Dense(units = 1, activation='sigmoid'))


In [8]:
# Compile the CNN
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total para

# Part2 : Fit the CNN to the images

In [10]:
# Step1: preprocessing the images
# Before fit the  model, proceed to image augmentation (consists of pre-processing images) to prevent overfitting
# data augmentation: random transformation on a random selection of images, like rotation, flipping, shifting, shearing...
# data augmentation functions: 1-enrich datasets without adding more images,
#                              2-reduce overfitting with a small amount of images

from keras.preprocessing.image import ImageDataGenerator

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)



In [12]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

# target_size = (64, 64), If want better performance, here can use big images, like 128,128
# the bigger the images are, the more information they can contain.

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [13]:
# Step2: fit and test the performance
model.fit_generator(training_set,
                    steps_per_epoch = 8000,
                    epochs = 5,
                    validation_data = test_set,
                    validation_steps = 2000)
# epochs = 5, cause my mac is sucks. It costs my mac pro the whole day to train 5 epoches.

Epoch 1/5
8000/8000 [==============================] - 2837s - loss: 0.3693 - acc: 0.8271 - val_loss: 0.4948 - val_acc: 0.8104
Epoch 2/5
8000/8000 [==============================] - 2811s - loss: 0.1408 - acc: 0.9448 - val_loss: 0.6677 - val_acc: 0.8280
Epoch 3/5
8000/8000 [==============================] - 3158s - loss: 0.0770 - acc: 0.9717 - val_loss: 0.7857 - val_acc: 0.8271
Epoch 4/5
8000/8000 [==============================] - 2824s - loss: 0.0562 - acc: 0.9799 - val_loss: 0.8147 - val_acc: 0.8323
Epoch 5/5
8000/8000 [==============================] - 2791s - loss: 0.0450 - acc: 0.9844 - val_loss: 0.9163 - val_acc: 0.8225


In [17]:
model.save('CNN2_catordog.h5')